# Submitting complex metadata

So far, in the previous notebooks, we have learned how to submit and perform operations on simple samples, that is; samples that are isolated in the submission, and all its metadata is just a bunch of simple values.

However, life is not as simple; sometimes, you will want to (and will have to, as seen in the notebook about sample validation) attach some extra metadata to your samples, that being how it relates to other samples (is it a replicate? is it derived from another sample that you are submitting or has been submitted?) or that extra bit of metadata that makes your fields very demure (e.g. specifying a unit, or ❤️referencing an ontology value❤️)

On this notebook, we will focus on exactly that, with the following TOC:

1. Before you start
2. Submitting samples with relationships
    1. All samples are being submitted for the first time
    2. Some samples are already accessioned; some are not
    3. All samples are accessioned but need to update relationships
3. How to improve your metadata via units and ontologies
4. Structured data
    1. WTH is structured data
    2. Structured data format and posting to sample
5. Posting a external URL (reference) to a sample

## Before you start

### Make sure you have biobroker >= 0.0.5
Before we start, we're going to do the same we always do; however, a lot of features for this notebook need `biobroker>=0.0.5` so... let's start with that!

In [1]:
%pip install biobroker==0.0.6 --upgrade

  Attempting uninstall: biobroker
    Found existing installation: biobroker 0.0.5
    Uninstalling biobroker-0.0.5:
      Successfully uninstalled biobroker-0.0.5
Note: you may need to restart the kernel to use updated packages.


(Please make sure you re-start the kernel if you upgraded your version!)

### Set-up and create samples
Once that's set-up, we're going to import everything and set up the basic metadata for 4 samples; the reason for setting up 4 samples this time will become apparent in the next sections

In [1]:
## Import everything we need
from biobroker.authenticator import WebinAuthenticator # Biosamples uses the WebinAuthenticator
from biobroker.api import BsdApi # BioSamples Database (BSD) API
from biobroker.metadata_entity import Biosample # The metadata entity
from biobroker.input_processor import TsvInputProcessor # An input processor
from biobroker.output_processor import XlsxOutputProcessor # An output processor
import os

## Generate sample
sample_tsv = [
    ["name", "collected_at", "organism"],
    ["Sample 1", "noon", "homo sapiens"],
    ["Sample 2", "noon", "homo sapiens"],
    ["Sample 3", "noon", "homo sapiens"],
    ["Sample 4", "noon", "homo sapiens"]
]

writable_sample = "\n".join(["\t".join(row) for row in sample_tsv])
with open("complex_sample_metadata.tsv", "w") as f:
    f.write(writable_sample)

path = "complex_sample_metadata.tsv" # This is the file we created previously

## Set up the required entities

input_processor = TsvInputProcessor(input_data=path)

os.environ['API_ENVIRONMENT'] = "dev" # There are multiple ways to set up environment variables

username = "" # Your username goes here
password = "" # Your password goes here
authenticator = WebinAuthenticator(username=username, password=password)

api = BsdApi(authenticator=authenticator)

2024-10-06 19:21:53,481 - BsdApi - INFO - Set up BSD API successfully: using base uri 'https://wwwdev.ebi.ac.uk/biosamples/samples'
2024-10-06 19:23:38,349 - Biosample - WARNING - Sample Sample 1: release date was not set. Setting it to right now.
2024-10-06 19:23:38,351 - Biosample - WARNING - Sample Sample 2: release date was not set. Setting it to right now.
2024-10-06 19:23:38,352 - Biosample - WARNING - Sample Sample 3: release date was not set. Setting it to right now.
2024-10-06 19:23:38,352 - Biosample - WARNING - Sample Sample 4: release date was not set. Setting it to right now.
2024-10-06 19:24:02,614 - Biosample - WARNING - Sample Sample 1: release date was not set. Setting it to right now.
2024-10-06 19:24:02,616 - Biosample - WARNING - Sample Sample 2: release date was not set. Setting it to right now.
2024-10-06 19:24:02,616 - Biosample - WARNING - Sample Sample 3: release date was not set. Setting it to right now.
2024-10-06 19:24:02,617 - Biosample - WARNING - Sample S

## Submitting samples with relationships

Depending on your metadata model, you may want to create entries in BioSamples that are related to each other; Maybe instead of repeating the important metadata from a parent sample (e.g. a cell line), you want to capture that metadata in its own entry and link the library preparations as samples derived from that one. 

Biosamples has got you covered! (And `biobroker` too). In BioSamples, you can link different samples that you have submitted using the attributes `derived_from`, `same_as`, `has_member` and `child_of`. You can find more information on these relationship types in https://www.ebi.ac.uk/biosamples/docs/guides/relationships. An important thing to remember about relationships is that they are **bi-directional**: That means, if you specify a type of relationship on sample A towards sample B, the inverse (or equal, in the case of `same_as`) will automatically apply on the sample

`biobroker` processes these relationships by reserving the name of the relationships as keywords for the metadata entry. What that means, in practice, is: to indicate that samples are related to each on the input metadata, you just need to add a column/field with the name of the keyword.

Let's go through a set of use cases so it becomes *clearer*:

### All samples are being submitted for the first time

Alright! So, let's say we have the four samples we have defined before. We know they are related in the following way:

```mermaid
graph LR;

A[Sample 1]
B[Sample 2]
C[Sample 3]
D[Sample 4]

B-->|derived_from|A
C-->|derived_from|A
D-->|derived_from|A
B-->|same_as|C
B-->|same_as|D
C-->|same_as|D
```

Looks complicated, but all I want to represent is: All samples are derived from `Sample 1`, and the derived samples are all replicates.

In this specific case, we have 4 samples that have not yet been submitted; and biosamples only accepts sample accessions as an input for the relationships. How are we going to do it?

The short and sad answer is: You need to submit the samples and then update them with the relationships. 

The cooler answer is that I got this covered in `biobroker`! On the `BsdApi` object there is a keyword argument you can pass to the `submit` function, `process_relationships`, that when set to True, will do the trick for you so you just need to worry.

But forget about all this theory and let's go into practice. The first thing we need to do is to define this relationships. You may have noticed we didn't `process` the samples before; that is because I want a clean slate for each of the examples, so we will start each example by loading the samples.

In [9]:
samples_relationships_non_accessioned = input_processor.process(Biosample)

Now that we have the samples loaded in, let's add the relationships. Since they are not accessioned, we are going to add the relationships using the name of the samples and the reserved keywords we talked about before. 

**Please note**: This is much easier and intuitive to do on the input metadata (excel, tsv etc) directly and load it with an input processor rather doing it here, but I am not going to create a file just for these samples. Just rembember that, index-wise, sample correspond to their index + 1 (e.g. index 0 of the samples is Sample 1)

In [10]:
## All samples are derived from Sample 1
samples_relationships_non_accessioned[1]['derived_from'] = "Sample 1"
samples_relationships_non_accessioned[2]['derived_from'] = "Sample 1"
samples_relationships_non_accessioned[3]['derived_from'] = "Sample 1"

## Samples 2, 3 and 4 are replicates.
samples_relationships_non_accessioned[1]['same_as'] = "Sample 3||Sample 4"
samples_relationships_non_accessioned[2]['same_as'] = "Sample 4"

Now we have the metadata in order to start submitting and processing those relationships!

One very important thing to notice is how I used the double pipes to indicate more than one relationship. You may need to adjust the delimiter for the different entities, depending on how you set the `Biosample`. The default delimiter for the entity is a double pipe (`||`). We will use this for multiple relationships, multiple urls, etc. It's not a perfect solution but it's a much easier to handle solution rather than starting to create columns with programmatic names and parsing that as arrays!

We will now submit the samples, as usual, with a slight change: we are going to provide with the `process_relationships` keyword. What this will do, in short, is to submit the samples as is, process the relationships afterwards, and update them.

In [11]:
submitted_samples = api.submit(samples_relationships_non_accessioned, process_relationships=True)

Let's see what our submitted samples look like!

In [13]:
print(f"https://wwwdev.ebi.ac.uk/biosamples/samples/{submitted_samples[0].accession}")

https://wwwdev.ebi.ac.uk/biosamples/samples/SAMEA131399004


![Parent sample](samples_relationship_parent.png)

As we can see... The relationships have gone through! Wooho! Now we have all of our samples linked together. This is just the example for the parent sample, but if you go to the link, you will see that the relationships are also defined for the child samples.

### Mixing accessioned and non-accessioned samples

The previous example was very simple; in that case, all of our samples were not submitted, so we could define all the relationships in our input and let `BsdApi.submit()` handle the process. However, that is not going to be always the case; sometimes you will have a parent sample that you have already registered, and child samples that you produced (replicates) that need to be now registered after you've finished your experiments.

For that, we are going to use the same set of samples as before, with the new relationships; just this time, we are going to submit the parent sample first.

In [2]:
samples_relationships_mixed_accessioned = input_processor.process(Biosample)
parent_sample = [samples_relationships_mixed_accessioned[0]]
child_samples = samples_relationships_mixed_accessioned[1:]

parent_sample_submitted = api.submit(parent_sample)

Once the parent sample is submitted, we are going to indicate the relationship in the child samples by **using the accession**

In [3]:
## All samples are derived from Sample 1
child_samples[0]['derived_from'] = parent_sample_submitted[0].accession
child_samples[1]['derived_from'] = parent_sample_submitted[0].accession
child_samples[2]['derived_from'] = parent_sample_submitted[0].accession

## Samples 2, 3 and 4 are replicates.
child_samples[0]['same_as'] = "Sample 3||Sample 4"
child_samples[1]['same_as'] = "Sample 4"

Now that we have our child samples set up, let's try to submit them!

In [4]:
submitted_samples_mixed_accessioning = api.submit(child_samples, process_relationships=True)

In [5]:
print(f"https://wwwdev.ebi.ac.uk/biosamples/samples/{submitted_samples_mixed_accessioning[0].accession}")

https://wwwdev.ebi.ac.uk/biosamples/samples/SAMEA131399017


And, as we can see, the samples are submitted with the relationships attached! Isn't that neat?

### Updating relationships

Updating the relationships for already accesioned samples is pretty straight-forward; for this, you just need to retrieve the samples (You can use multiple methods for that - You can refer to notebook 4 for more information).

Once you've got the samples, you just need to update the metadata to include the relationship; for this, you can either:
* Add the value for each of the samples retrieved in your script and update them using `BsdApi.update()`
* Output the samples to a more user-friendly format (Xlsx, tsv) modify them, load them back and update them using `BsdApi.update()`

The metadata entities automatically recognise that the origin sample is accessioned, so when you add the relationship, it will be added to the proper metadata and you don't need to use the `process_relationships` function.

There is a third, more complicated option, which is to specify them somewhat manually; there is a function, [`add_relationship`](https://biobroker.readthedocs.io/en/latest/biobroker.metadata_entity.html#biobroker.metadata_entity.Biosample.add_relationship) for which you can specify the relationship manually, but... I don't really think it's worth it. I made it really simple for the user!

## Adding ontology and unit tags to your values

Another cool thing that BSD has, is that for each property, you can specify a `unit` or an `ontology term`, so that the term contains a bit more extra implicit metadata without tampering with the field name (**tip**: A field name should not contain the unit of measurement! That's usually bad practice!!)

Let's imagine that you have measured the depth of some soil samples that you have taken. This depth was measured in meters, and you made sure your samples got taken at 1 meter, since it's the best depth ever for these samples (Please don't hate me I know nothing about soil)

You want to use the GSC MIxS soil checklist (ERC000020), and you see the `depth` field to collect this information. There is a slight problem; you don't see where to input the units. You see in other submitted samples that it is indeed specified, but there is a parenthesis encasing the unit. What is going on here?

BSD has defined its own way to deal with units and ontology terms; instead of having 2 fields and linking them, they have the information defined within the field. For context, a field in BSD usually looks like this:
```json
...
<field_name>: [
    {
        "text": <value_of_field>
    }
]
...
```

(Now you're happy you don't have to deal with that when using this library, eh?)

Adding a unit is as simple as this:

```json
...
<field_name>: [
    {
        "text": <value_of_field>,
        "unit": <unit_name>
    }
]
...
```

And it's even simpler with the library. For the BioSample metadata entity, a delimiter is defined; that delimiter, aside from allowing to specify more than one relationship in the same field, it also allows you to define the unit in the metadata. Let's use the `depth` example so that it is clearer!

In [3]:
samples_units = input_processor.process(Biosample)

# Let's define the depth in the first sample and see what it looks like

samples_units[0]['depth'] = 1

print(samples_units[0].entity)

{'name': 'Sample 1', 'characteristics': {'collected_at': [{'text': 'noon'}], 'organism': [{'text': 'homo sapiens'}], 'depth': [{'text': 1}]}, 'release': '2024-10-06T18:24:02.616045Z'}


In [4]:
# Now, let's add the unit! This also works for the headers of the user-friendly inputs (Like excel and tsv)
samples_units[0]['depth||unit'] = 'meter'

print(samples_units[0].entity)

{'name': 'Sample 1', 'characteristics': {'collected_at': [{'text': 'noon'}], 'organism': [{'text': 'homo sapiens'}], 'depth': [{'text': 1, 'unit': 'meter'}]}, 'release': '2024-10-06T18:24:02.616045Z'}


And Ta-da! we've got our sample with the `unit` attached! At this point, you can submit it to BSD and it will appear properly. As I mention in the code, you can do this in a programmatic way or you can add your units and/or ontology terms (For more information on the tags, you can read the [BSD documentation on submitting a sample](https://www.ebi.ac.uk/biosamples/docs/references/api/submit) (However, on the tags, you'll see there's only 3: 'unit', 'text' and 'ontologytTerms')

## Structured data

### What in the world is structured data

There is another layer of information that you can add to a sample; when you're recording metadata for a sample, you can sometimes generate a table of information (e.g. an Antibiogram). You could attach this information to the sample in a very weird and complicated way; however, biosamples provides with the ability to provide tables with tagged information and a `title`, and they call it `structured data`. For a couple examples of structured data, you can look at this sample in production: https://www.ebi.ac.uk/biosamples/samples/SAMEA112948612 or look at the [documentation examples](https://www.ebi.ac.uk/biosamples/docs/references/api/submit#_submit_structured_data)

### Structured data format

Let's do a simple task of creating some very simple structured data. Let's say that, for our case, we have realised an assay on the sample, that returns a very complex 2x2 table (scary)

For this, let's start by loading up the samples and getting just the first one for this task and define the table!

In [6]:
# Get the samples
samples_st = input_processor.process(Biosample)

# Choose just the first sample and submit it
sample_example = [samples_st[0]]

sample_example_submitted = api.submit(sample_example)

data_table = [[1, 34.5],
              [2, 33.7]]

Now that we have all the necessary things set up, let's start structuring the table in the necessary format. In the code, this will all be one step, but take a look at the comments! I will be commenting each line that needs clarification.

In [11]:
structured_data = {  # Please note, the structured data is submitted as a dictionary!
    'accession': sample_example_submitted[0].accession,  # Accession must be provided with the structured data
    'data': [ # This array is for all the different, unrelated tables you want to submit. Here, we only have one table, so the array will have lenght 1
        {
            'webinSubmissionAccountId': 'Webin-64342',  # For some goddam reason they need your webin account there
            'type': 'MYAWESOMEASSAY', # This is the name of the table; usually, the name of the assay
            # Note here: You can specify a `schema` property to validate the content of your table. Not going to do it
            'content': [ # This is the actual data table! wooho! each of the elements of the aray is a dictionary with the values associated to a field name
                { # We have 2 rows, so we expect 2 dictionaries
                    'time point': { # Each field-value is defined as in the usual for BSD: 'value' is needed (instead of 'text'), and you can add tags
                        'value': 1,
                        'unit': 'hour'
                    },
                    'luminosity': {
                        'value': 34.5
                    }
                },
                { 
                    'time point': { 
                        'value': 2,
                        'unit': 'hour'
                    },
                    'luminosity': {
                        'value': 33.7
                    }
                }
            ]
        }
    ]
}

# You could also generate the content automatically by iterating the data table. Feel free to use your preferred method!
content = [{'timepoint': {'value': row[0], 'unit': 'hour'}, 'luminosity': {'value': row[1]}} for row in data_table] 
structured_data['data'][0]['content'] = content

# Let's take a look at the structured data!

print(structured_data)

{'accession': 'SAMEA131399379', 'data': [{'webinSubmissionAccountId': 'Webin-64342', 'type': 'MYAWESOMEASSAY', 'content': [{'timepoint': {'value': 1, 'unit': 'hour'}, 'luminosity': {'value': 34.5}}, {'timepoint': {'value': 2, 'unit': 'hour'}, 'luminosity': {'value': 33.7}}]}]}


In [15]:
response = api.authenticator.post(url=f"{api.structured_data_endpoint}/{structured_data['accession']}", payload=structured_data)
print(response.text)

{
  "accession" : "SAMEA131399379",
  "create" : "2024-10-06T19:18:11.785Z",
  "update" : "2024-10-06T19:18:11.786Z",
  "data" : [ {
    "domain" : null,
    "webinSubmissionAccountId" : "Webin-64342",
    "type" : "MYAWESOMEASSAY",
    "schema" : null,
    "content" : [ {
      "timepoint" : {
        "value" : "1",
        "iri" : null
      },
      "luminosity" : {
        "value" : "34.5",
        "iri" : null
      }
    }, {
      "timepoint" : {
        "value" : "2",
        "iri" : null
      },
      "luminosity" : {
        "value" : "33.7",
        "iri" : null
      }
    } ]
  } ]
}


(Another error I found while writing these examples - The returned response is the structured data, not the sample - will fix in 0.0.7 and use the proper function to do this)

As you can see, it has returned the data created. Let's take a look at the BioSamples entry!

https://wwwdev.ebi.ac.uk/biosamples/samples/SAMEA131399379

![BSD Structured data](bsd_data_entry_sd.png)

## Posting an external reference (URI)

The last, and probably easiest, is adding external references. In BSD, external references are used to link data that could not be hosted in BSD to the entry, to have a single `source of truth` for all the data related to a sample.

One example of this are the ENA links to the Genomics/Transcriptomics data - But in any case, these links are generated automatically by BioSamples if you remember to use your sample accession in your ENA submission :)

Here I will only say that, similarly to the relationships, URLs can be added with the function `add_external_reference()` on the samples.